In [8]:
!pip install torch torchvision torchaudio torch-geometric pandas


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.1/63.1 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 22.1 MB/s eta 0:00:0000:01


In [4]:
import zipfile
import os
import urllib.request

# Download the dataset
url = "https://files.grouplens.org/datasets/movielens/ml-latest-small.zip"
filename = "ml-latest-small.zip"
urllib.request.urlretrieve(url, filename)

# Extract the dataset
with zipfile.ZipFile(filename, 'r') as zip_ref:
    zip_ref.extractall("ml-latest-small")

# Load the CSV file
# df = pd.read_csv('/kaggle/working/ml-latest-small/ml-latest-small/ratings.csv')

In [5]:
import pandas as pd
df = pd.read_csv('/kaggle/working/ml-latest-small/ml-latest-small/ratings.csv')

In [6]:
df.head()

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931


In [9]:
import torch
import pandas as pd
from torch_geometric.data import Data
from torch_geometric.nn import GATConv

# Load Movielens dataset (ratings data)
# url = "https://files.grouplens.org/datasets/movielens/ml-latest-small.zip"
# df = pd.read_csv('/kaggle/input/ml-latest-small/ratings.csv')

# Extract necessary columns: userId, movieId, rating
df = df[['userId', 'movieId', 'rating']]

# Map users and movies to consecutive node IDs
user_mapping = {id: i for i, id in enumerate(df['userId'].unique())}
movie_mapping = {id: i + len(user_mapping) for i, id in enumerate(df['movieId'].unique())}

df['userId'] = df['userId'].map(user_mapping)
df['movieId'] = df['movieId'].map(movie_mapping)

# Create edge index (graph structure) and edge features (ratings)
edge_index = torch.tensor([df['userId'].values, df['movieId'].values], dtype=torch.long)
edge_attr = torch.tensor(df['rating'].values, dtype=torch.float)

# Define the number of user nodes and movie nodes
num_users = len(user_mapping)
num_movies = len(movie_mapping)
num_nodes = num_users + num_movies


/tmp/ipykernel_30/208465854.py:21: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:278.)
  edge_index = torch.tensor([df['userId'].values, df['movieId'].values], dtype=torch.long)


In [10]:
import torch.nn.functional as F
from torch_geometric.nn import GATConv

class GATRecommendation(torch.nn.Module):
    def __init__(self, in_channels, out_channels, heads=1, dropout=0.3):
        super(GATRecommendation, self).__init__()
        
        # First GAT layer
        self.gat1 = GATConv(in_channels, out_channels, heads=heads, dropout=dropout)
        
        # Second GAT layer
        self.gat2 = GATConv(out_channels * heads, out_channels, heads=heads, concat=False, dropout=dropout)

        # Classifier for user-movie interaction prediction
        self.classifier = torch.nn.Linear(out_channels, 1)
        
    def forward(self, x, edge_index, edge_attr):
        # Strong and weak ties (attention coefficients depend on edge_attr, i.e., ratings)
        x = F.elu(self.gat1(x, edge_index, edge_attr=edge_attr))
        x = self.gat2(x, edge_index, edge_attr=edge_attr)
        
        return x
    
    def predict(self, x):
        return torch.sigmoid(self.classifier(x))


In [11]:
import torch
import torch_geometric
from torch_geometric.data import Data

# Assuming you have 610 users and 9724 movies
num_users = 610
num_movies = 9724
total_nodes = num_users + num_movies

# Initialize node features (one-hot encoding)
user_features = torch.eye(num_users)  # Shape: [610, 610]
movie_features = torch.eye(num_movies)  # Shape: [9724, 9724]

# Pad user features to match movie feature dimension and vice versa
user_features_padded = torch.cat([user_features, torch.zeros(num_users, num_movies)], dim=1)  # Shape: [610, 9724 + 610]
movie_features_padded = torch.cat([torch.zeros(num_movies, num_users), movie_features], dim=1)  # Shape: [9724, 9724 + 610]

# Combine user and movie features into a single feature matrix
x = torch.cat([user_features_padded, movie_features_padded], dim=0)  # Shape: [610 + 9724, 610 + 9724]

# Print the size of the combined feature matrix
print(x.shape)  # Should print: torch.Size([10334, 10334])


torch.Size([10334, 10334])


In [12]:
# Assuming edge_index and edge_attr are already defined (they represent the graph structure)
data = Data(x=x, edge_index=edge_index, edge_attr=edge_attr)

# Check the data object
print(data)


Data(x=[10334, 10334], edge_index=[2, 100836], edge_attr=[100836])


In [13]:
import torch.optim as optim

# Define the model, optimizer, and loss function
model = GATRecommendation(in_channels=data.num_features, out_channels=64, heads=4, dropout=0.3)
optimizer = optim.Adam(model.parameters(), lr=0.005, weight_decay=5e-4)
criterion = torch.nn.BCELoss()

# Split data into training and testing sets (80-20 split)
train_mask = torch.rand(data.num_edges) < 0.8
test_mask = ~train_mask

# Helper function to get edge predictions
def get_edge_predictions(node_embeddings, edge_index):
    # Get source and target node embeddings for each edge
    source_embeddings = node_embeddings[edge_index[0]]
    target_embeddings = node_embeddings[edge_index[1]]
    
    # Compute edge predictions (e.g., inner product or similarity between node embeddings)
    edge_predictions = torch.sigmoid((source_embeddings * target_embeddings).sum(dim=1))
    
    return edge_predictions

# Training the GAT model
def train():
    model.train()
    optimizer.zero_grad()

    # Forward pass: Get node embeddings from the model
    out = model(data.x, data.edge_index, data.edge_attr)

    # Get edge predictions for the training set
    train_edge_predictions = get_edge_predictions(out, data.edge_index[:, train_mask])

    # Ensure that edge attributes are between 0 and 1 (if binary)
    data.edge_attr = torch.clamp(data.edge_attr, 0, 1)

    # Compute the loss on the training edges
    loss = criterion(train_edge_predictions, data.edge_attr[train_mask].float())

    # Backward pass and optimization
    loss.backward()
    optimizer.step()

    return loss.item()


# Testing the GAT model
def test():
    model.eval()
    with torch.no_grad():
        # Get node embeddings from the model
        out = model(data.x, data.edge_index, data.edge_attr)
        
        # Get edge predictions for the test set
        test_edge_predictions = get_edge_predictions(out, data.edge_index[:, test_mask])
        
        # Compute the loss on the test edges (no unsqueeze needed here)
        loss = criterion(test_edge_predictions, data.edge_attr[test_mask].float())
        
    return loss.item()

# Training loop
for epoch in range(200):
    loss = train()
    if epoch % 10 == 0:
        test_loss = test()
        print(f'Epoch {epoch}, Loss: {loss}, Test Loss: {test_loss}')


Epoch 0, Loss: 0.6929823160171509, Test Loss: 0.690059244632721
Epoch 10, Loss: 0.36320239305496216, Test Loss: 0.5957578420639038
Epoch 20, Loss: 0.6578162908554077, Test Loss: 0.6523407697677612
Epoch 30, Loss: 0.5867574214935303, Test Loss: 0.6105227470397949
Epoch 40, Loss: 0.1783255785703659, Test Loss: 0.11204014718532562
Epoch 50, Loss: 0.048870399594306946, Test Loss: 0.03552335500717163
Epoch 60, Loss: 0.040481775999069214, Test Loss: 0.03467473387718201
Epoch 70, Loss: 0.04163932055234909, Test Loss: 0.03444087132811546
Epoch 80, Loss: 0.040608689188957214, Test Loss: 0.03381260111927986
Epoch 90, Loss: 0.03956958279013634, Test Loss: 0.033914897590875626
Epoch 100, Loss: 0.03966621309518814, Test Loss: 0.033706068992614746
Epoch 110, Loss: 0.03881607577204704, Test Loss: 0.033678848296403885
Epoch 120, Loss: 0.0375114269554615, Test Loss: 0.03372076153755188
Epoch 130, Loss: 0.03825031593441963, Test Loss: 0.0336945503950119
Epoch 140, Loss: 0.038477249443531036, Test Loss: 